In [1]:
import numpy as np
import pandas as pd
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

import sys
import operator

import fastparquet
import snappy

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import CategoricalIndex, RangeIndex, Index, MultiIndex


In [15]:
alldays = pd.read_csv('VehtoSep11.csv')

In [20]:
alldays.loc[40:45]

,Begin Date,Begin Time,End Time,Vehicle Location,Vehicle Type,Vehicle Characteristics,Transaction Characteristics,Bikeway Users Displaced,LZ Space Avail,Occupied while idle?,CNS?,TNC?,Notes,Duration
40,8/21/2019,1:34:37 PM,1:42:40 PM,Space 2,Car,Other significant characteristic (see note),No interaction with parking meter,NaN,NaN,False,True,False,A passenger got out and the car moved to space...,0:08:03
41,8/21/2019,1:36:07 PM,1:38:23 PM,Space 1,Car,Other significant characteristic (see note),No interaction with parking meter,NaN,NaN,False,True,False,Passenger got out and brought bags back,0:02:16
42,8/21/2019,1:40:25 PM,1:42:10 PM,Space 1,Car,NaN,No interaction with parking meter,NaN,NaN,False,True,False,NaN,0:01:45
43,8/21/2019,1:45:10 PM,3:19:09 PM,Space 3,Car,NaN,NaN,NaN,2.0,False,False,False,NaN,1:33:59
44,8/21/2019,1:48:57 PM,2:45:22 PM,Space 2,Car,NaN,No interaction with parking meter,NaN,NaN,False,True,False,Passenger got out after parking for 12 minutes...,0:56:25
45,8/21/2019,1:49:57 PM,1:50:10 PM,SB bike lane,Car,NaN,NaN,0.0,1.0,True,False,False,NaN,0:00:13


In [8]:
# alldays.dropna(subset=['Begin Date'], inplace=True)
# alldays['Begin Date'] = alldays['Begin Date'].apply(zeropad_dates)
# alldays[alldays['Begin Date'] == '08/30/2019'].to_csv('Aug30.csv')

In [2]:
def showall(df):
    #shows entire dataframe
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)

In [3]:
def zeropad_dates(str_date):
    ##TODO speedup candidate?
    #pads month and day with zeroes
    padded_date = ''
    split_date = str_date.split('/')
    for value in split_date:
        if len(value) < 2:
            value = '0' + value
        padded_date += (value + '/')
    padded_date = padded_date[:-1]
    return padded_date

In [4]:
def datetime_from_str(string):
    #converts string format dates/times from spreadsheet into Python Datetime objects
    
    try:
        #print('ran, string:{}'.format(string))
        format_str = '%m/%d/%Y%I:%M:%S %p'
        to_dt = dt.datetime.strptime(string, format_str)
    except ValueError:
        #handles small portion of values in 24hr format
        try:
            #print('excepted, string:{}'.format(string))
            format_str = '%m/%d/%Y%H:%M:%S'
            to_dt = dt.datetime.strptime(string, format_str)
        
        except ValueError:
            #use dict to handle both text-described times
            text_times_dict = {'before 12':'12:00:00', 'end of the day':'22:30:00'}
            #print('excepted2, string:{}'.format(string))
            texttime = string[10:]
            time = text_times_dict[texttime]
            combined = string[:10] + time
            return datetime_from_str(combined)
    return to_dt

In [5]:
def classify_violator(row, st_time='18:00:00', end_time='22:00:00', legal_minutes=15):
    ##TODO speedup candidate?
    ##TODO rewrite cleaner?
    
    #classify a row as a violator or not
    #considers +15min parked from 18:00-22:30 a violation
    
    begin, end = row['Begin Date']+ row['Begin Time'], row['Begin Date'] + row['End Time']
    time_parked = {'Start':datetime_from_str(begin), 'End':datetime_from_str(end)}
    datestr = begin[:10]
    
    enf_start = datestr + st_time
    enf_end = datestr + end_time
    enforcement_times = {'Start':datetime_from_str(enf_start), 'End':datetime_from_str(enf_end)}
    
    latest_start = max(time_parked['Start'], enforcement_times['Start'])
    earliest_end = min(time_parked['End'], enforcement_times['End'])
    delta = (earliest_end - latest_start).seconds
    
    #maximum parking duration during enforcement interval
    legal_duration_seconds = legal_minutes*60
    max_observation = 60**2 * 11
    if (delta > legal_duration_seconds 
        and delta < max_observation 
        #Space 3 not part of loading zone
        and row['Vehicle Location'] in ['Space 1', 'Space 2']):
        
        return 'Violator'
    elif row['Vehicle Location'] == 'SB bike lane':
        return 'Bike Lane Blocking'
    else:
        return 'Likely Non-Violator' 
    
def classify_violators(df):
    #add boolean violator column to original (pre-timestamp) df. Also zeropads dates.
    
    df.dropna(subset=['Begin Time', 'End Time'], inplace=True)
    df['Begin Date'] = df['Begin Date'].apply(zeropad_dates)
    df['Violator'] = df.apply(classify_violator, axis=1)
    
    return df


In [27]:
# ##TODO remove unnecessary multiindex...
# def timestamps_from_interval(dt_start, dt_end):
#     ##TODO speedup, may be tough to speedup datetime comparison... 
#     #generates list of datetime objects for every second in interval
#     timestamps = [dt_start]
#     timestamp = dt_start
#     while timestamp < dt_end:
#         timestamp += dt.timedelta(seconds=1)
#         timestamps = timestamps + [timestamp]
#     return timestamps

# def timestamps_from_row(row):
#     #extracts interval from a row of the original spreadsheet
#     start = row[1]['Begin Date'] + row[1]['Begin Time'] 
#     end = row[1]['Begin Date'] + row[1]['End Time'] 
    
#     dt_start = datetime_from_str(start)
#     dt_end = datetime_from_str(end)

#     return timestamps_from_interval(dt_start, dt_end)

# def multiindex_for_row(row):
#     #creates multiindex of timestamps and vehicle locations
#     timestamps = timestamps_from_row(row)
#     location = [row[1]['Vehicle Location']]
#     locations = location * len(timestamps)
#     tuples = list(zip(timestamps, locations))
    
#     return pd.MultiIndex.from_tuples(tuples, names=['Timestamp', 'Vehicle_Location'])
    
# def df_for_row(row):
#     #generates multi-indexed (timestamped) dataframe from a single row of the original spreadsheet
#     multiind = multiindex_for_row(row)
#     row_df = pd.DataFrame(row[1]).swapaxes('index', 'columns')
#     row_df = pd.concat([row_df]*len(multiind))
#     row_df.index = multiind
#     return row_df

In [17]:
##Testversion 
##TODO remove unnecessary multiindex...
def timestamps_from_interval(dt_start, dt_end):
    ##TODO speedup, may be tough to speedup datetime comparison... 
    #generates list of datetime objects for every second in interval
    timestamps = [dt_start]
    timestamp = dt_start
    while timestamp < dt_end:
        timestamp += dt.timedelta(seconds=1)
        timestamps = timestamps + [timestamp]
    return timestamps

def timestamps_from_row(row):
    #extracts interval from a row of the original spreadsheet
    start = row[1]['Begin Date'] + row[1]['Begin Time'] 
    end = row[1]['Begin Date'] + row[1]['End Time'] 
    
    dt_start = datetime_from_str(start)
    dt_end = datetime_from_str(end)

    return timestamps_from_interval(dt_start, dt_end)

def index_for_row(row):
    #creates timestamp index
    timestamps = timestamps_from_row(row)
    return pd.Index(timestamps, name='Timestamp')
    
def df_for_row(row):
    #generates multi-indexed (timestamped) dataframe from a single row of the original spreadsheet
    ind = index_for_row(row)
    row_df = pd.DataFrame(row[1]).swapaxes('index', 'columns')
    row_df = pd.concat([row_df]*len(ind))
    row_df.index = ind
    return row_df

In [13]:
def timestamp_df(df):
    ##TODO speedup, probably tough to get rid of the for loop here
    #Generates timestamped dataframe from original spreadsheet, after classify violators. Currently slow to run. 
    timestamped_df = pd.DataFrame()
    i = 0
    for row in df.iterrows():
        if i % 100 == 0:
            #TODO rewrite to include %complete, ETA?
            print('Processing Row: {}, {}% complete'.format(i, int((i/df.shape[0])*100)))
        i += 1
        timestamped_df = timestamped_df.append(df_for_row(row))
    return timestamped_df

In [14]:
def timestamp_and_classify_violators(df):
    return timestamp_df(classify_violators(df)) 

In [9]:
def reset_viol_time(df, hour=22, opr='=='):
##TODO reduce reduncency? 

    ops = {'>': operator.gt,
           '<': operator.lt,
           '>=': operator.ge,
           '<=': operator.le,
           '==': operator.eq}

    df.sort_index(level='Timestamp', inplace=True)
    times = df.index.get_level_values(level='Timestamp')
    hr = ops[opr](times.hour, hour)
    filtered = df.loc[hr]
    filtered.loc[:, 'Violator'] = filtered.loc[:, 'Violator'].apply(lambda x: 'Likely Non-Violator' if x == 'Violator' else x)
    df = df.loc[~hr]
    df = df.append(filtered)
    return df

##TODO add to initial viol classifier
def add_tnc_cns(df):
    if df['CNS?']:
        df['Violator'] = df['Violator'] + ', CNS'
    elif df['TNC?']:
        df['Violator'] = df['Violator'] + ', TNC'
    return df
#%prun -l 20 time alldays_timestamped = alldays_timestamped.apply(add_tnc_cns, axis=1)

In [28]:
alldays_timestamped = timestamp_and_classify_violators(alldays)

Processing Row: 0, 0% complete
Processing Row: 100, 4% complete
Processing Row: 200, 9% complete
Processing Row: 300, 13% complete
Processing Row: 400, 18% complete
Processing Row: 500, 22% complete
Processing Row: 600, 27% complete
Processing Row: 700, 31% complete
Processing Row: 800, 36% complete
Processing Row: 900, 40% complete
Processing Row: 1000, 45% complete
Processing Row: 1100, 49% complete
Processing Row: 1200, 54% complete
Processing Row: 1300, 58% complete
Processing Row: 1400, 63% complete
Processing Row: 1500, 67% complete
Processing Row: 1600, 72% complete
Processing Row: 1700, 76% complete
Processing Row: 1800, 81% complete
Processing Row: 1900, 85% complete
Processing Row: 2000, 90% complete
Processing Row: 2100, 94% complete
Processing Row: 2200, 99% complete


In [29]:
alldays_timestamped = reset_viol_time(alldays_timestamped)
alldays_timestamped = reset_viol_time(alldays_timestamped, hour=18, opr='<')
alldays_timestamped = alldays_timestamped.apply(add_tnc_cns, axis=1)

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/pandas/core/indexing.py:1046: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [30]:
alldays_timestamped.shape

(1569887, 15)

In [16]:
alldays_timestamped.shape

(1569887, 15)

In [31]:
alldays_timestamped.loc['2019-09-01 18:31:48']

,Begin Date,Begin Time,End Time,Vehicle Location,Vehicle Type,Vehicle Characteristics,Transaction Characteristics,Bikeway Users Displaced,LZ Space Avail,Occupied while idle?,CNS?,TNC?,Notes,Duration,Violator
Timestamp,,,,,,,,,,,,,,,
2019-09-01 18:31:48,09/01/2019,5:00:00 PM,6:32:42 PM,Space 3,Car,NaN,NaN,NaN,NaN,False,False,False,NaN,1:32:42,Likely Non-Violator
2019-09-01 18:31:48,09/01/2019,6:31:10 PM,6:33:20 PM,SB bike lane,Car,Blinkers on,NaN,6.0,0.0,True,False,False,NaN,0:02:10,Bike Lane Blocking
2019-09-01 18:31:48,09/01/2019,6:28:52 PM,6:31:53 PM,NB right turn pocket,Car,NaN,NaN,NaN,0.0,False,True,False,NaN,0:03:01,"Likely Non-Violator, CNS"
2019-09-01 18:31:48,09/01/2019,6:16:07 PM,9:59:08 PM,Space 1,Car,NaN,Interaction with parking meter,NaN,NaN,False,False,False,looked at the meter but still decided to park,3:43:01,Violator
2019-09-01 18:31:48,09/01/2019,6:23:41 PM,7:43:29 PM,Space 2,Car,NaN,No interaction with parking meter,NaN,NaN,False,False,False,NaN,1:19:48,Violator


In [19]:
# alldays_timestamped.xs('2019-09-01 18:31:48', level='Timestamp', drop_level=False)

Begin Date  Begin Time    End Time  \
Timestamp           Vehicle_Location                                           
2019-09-01 18:31:48 NB right turn pocket  09/01/2019  6:28:52 PM  6:31:53 PM   
                    SB bike lane          09/01/2019  6:31:10 PM  6:33:20 PM   
                    Space 1               09/01/2019  6:16:07 PM  9:59:08 PM   
                    Space 2               09/01/2019  6:23:41 PM  7:43:29 PM   
                    Space 3               09/01/2019  5:00:00 PM  6:32:42 PM   

                                              Vehicle Location Vehicle Type  \
Timestamp           Vehicle_Location                                          
2019-09-01 18:31:48 NB right turn pocket  NB right turn pocket          Car   
                    SB bike lane                  SB bike lane          Car   
                    Space 1                            Space 1          Car   
                    Space 2                            Space 2          Car   
                    Space 3                            Space 3          Car   

                                         Vehicle Characteristics  \
Timestamp           Vehicle_Location                               
2019-09-01 18:31:48 NB right turn pocket                     NaN   
                    SB bike lane                     Blinkers on   
                    Space 1                                  NaN   
                    Space 2                                  NaN   
                    Space 3                                  NaN   

                                                Transaction Characteristics  \
Timestamp           Vehicle_Location                                          
2019-09-01 18:31:48 NB right turn pocket                                NaN   
                    SB bike lane                                        NaN   
                    Space 1                  Interaction with parking meter   
                    Space 2               No interaction with parking meter   
                    Space 3                                             NaN   

                                         Bikeway Users Displaced  \
Timestamp           Vehicle_Location                               
2019-09-01 18:31:48 NB right turn pocket                     NaN   
                    SB bike lane                               6   
                    Space 1                                  NaN   
                    Space 2                                  NaN   
                    Space 3                                  NaN   

                                         LZ Space Avail Occupied while idle?  \
Timestamp           Vehicle_Location                                           
2019-09-01 18:31:48 NB right turn pocket              0                False   
                    SB bike lane                      0                 True   
                    Space 1                         NaN                False   
                    Space 2                         NaN                False   
                    Space 3                         NaN                False   

                                           CNS?   TNC?  \
Timestamp           Vehicle_Location                     
2019-09-01 18:31:48 NB right turn pocket   True  False   
                    SB bike lane          False  False   
                    Space 1               False  False   
                    Space 2               False  False   
                    Space 3               False  False   

                                                                                  Notes  \
Timestamp           Vehicle_Location                                                      
2019-09-01 18:31:48 NB right turn pocket                                            NaN   
                    SB bike lane                                                    NaN   
                    Space 1               looked at the meter but still decided to park

In [32]:
alldays_timestamped.to_parquet('TimestampToSep11.parquet')